# Resumo do código

### <u>Código que gera o ficheiro OUT_1_DataFusion</u>
---
O objectivo é receber dados dos Ninjas e da Delta e devolver um conjunto de métricas para estudar o comportamento de vários produtos em diversas lojas. 

---
- Inputs

> __Dados dos ninjas__ em formato Wide
> - 1's e 0's consoante a presença e ausência do produto, Data e Loja

> __Dados da Delta__ em formato Long
> - Stocks e trânsito, Sellout dia anterior

- Outputs

> __Ficheiro Long__ (artigos na mesma coluna)

> __Métricas novas:__
> - Sinal
> - Ciclos e Adequação de Stock
> - Restock


In [1]:
import nbformat

notebook_A = nbformat.read("C:\\Users\\joao_\\OneDrive\\Ambiente de Trabalho\\B&N Código\\Delta\\cPadrão\\cPadrão_05-Maio_23\\0PMaio_DataCleaning.ipynb", nbformat.NO_CONVERT)
notebook_B = nbformat.read("C:\\Users\\joao_\\OneDrive\\Ambiente de Trabalho\\B&N Código\\Delta\\cPadrão\\cPadrão_05-Maio_23\\1PMaio_Wrangling_Long.ipynb", nbformat.NO_CONVERT)

notebook_A.cells.extend(notebook_B.cells)

nbformat.write(notebook_A, "merged_notebook.ipynb")

In [1]:
import pandas as pd
import numpy as np

def escrever_excel(dataFrame, nomeFicheiro):
    dataFrame.to_excel('%s.xlsx' %nomeFicheiro, index=False)

- # `Ninjas`

In [11]:
#Info Ninja

dfNinjas=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão_05_2023\\1Ninjas_Limpo.xlsx")

CPU times: total: 31.2 ms
Wall time: 45.1 ms


> Definir variáveis importantes
> - Se houver um ficheiro com os produtos

In [12]:
# Ler ficheiro para dataframe
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Padrão_05_2023\\produtos.txt', header=None)

# Passar para uma lista
produtos = df_produtos[0].tolist()
resto = dfNinjas.columns.difference(produtos)

> - Se se quiser usar o primeiro e o último produtos

In [7]:
# Nomes de entrada
x= input("Primeiro produto: ")                # CAFÉ DELTA Q MYTHIQ 80CAP
y= input("Último produto: ")                  # Delta Cafés Mistura + café 200grs

# Passar para uma lista
produtos = dfNinjas.columns[dfNinjas.columns.get_loc(x): dfNinjas.columns.get_loc(y)+1].tolist()
resto = dfNinjas.columns.difference(produtos)

Primeiro produto: CAFÉ DELTA Q MYTHIQ 80CAP
Último produto: Delta Cafés Mistura + café 200grs


> Reposição

In [26]:
# Ler ficheiro
dfRepos=pd.read_excel("D:\\B&N Dados\\Delta\\Reposição_Sonae_Código.xlsx")

# Criar coluna de reposição
dfNinjas['Reposição'] = [1 if val in dfRepos['STORE'].values else 0 for val in dfNinjas['STORE']]

> Vendedor

In [23]:
# Ler ficheiro
dfVendedor=pd.read_excel("D:\B&N Dados\Delta\Padrão_05_2023\\Vendedor.xlsx", sheet_name = "Lista Lojas Sonae")
dfVendedor = dfVendedor.rename(columns={"Cód. Loja":"STORE"})

# Criar coluna de reposição
dfNinjas = pd.merge(dfNinjas, dfVendedor[["STORE","Vendedor"]], how="left", on = "STORE")

- # `Delta`

In [4]:
# Ler o ficheiro long com Stocks e Fornecimento
dfDelta=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão_05_2023\\2_Delta.xlsx", sheet_name="Sheet1")

# Renomear
dfDelta = dfDelta.drop(columns="STORE_NAME")

## O que tem cada ficheiro:

### **dfDelta**:
1. Formato long (produtos numa única coluna "DESC_ARTIGO")
2. Stocks, em Trânsito, Esperado, Linear, sellout dia anterior	


### **dfNinjas**:
1. Formato wide (cada produto numa coluna)
2. Presença de produto em linear para cada produto

### **dfSonae**:
1. Formato wide
2. Sellouts, Vendedores, Reposição


#  <span style="color:green"><u> Mergir</u> </span>

In [9]:
# Passar ficheiro ninjas para long
dfNinjasLong = dfNinjas.melt(id_vars=resto, value_vars=produtos, var_name='DESC_ARTIGO', value_name='NinjaInfo')


#Mergir
dfMeio = pd.merge(dfNinjasLong, dfDelta, how="left", on = ["DATA","STORE", "DESC_ARTIGO"]) 
dfMeio = dfMeio.drop(columns=["Loja"])

In [ ]:
dfFinal=dfMeio.copy()

###  <span style="color:green">Fim</span>

# Já temos DataFrame com:
- Informação Ninjas
- Informação Stocks, Linear, Intransit e Expected
- Informação Sellouts, Vendedor, Reposição

## Fazer:

- Rotura
- Sinal -> Avaliação da informação ninja vs stock
- Ciclos
- Adequação de Stock

> STK

In [ ]:
# Definir coluna de STK (soma de stocks com stock em trânsito)

dfFinal["STK"] = dfFinal["STOCK"] + dfFinal["INTRANSIT"] + dfFinal["EXPECTED"]

> Balanço

In [ ]:
# Definir coluna de Balanço (razão entre sellout e stock total)

dfFinal["Balanço"] = dfFinal["SELLOUT"] / dfFinal["STK"]

> Rotura

In [15]:
# Definir coluna de rotura (se stock menor ou igual a 0 e existe Linear)

dfFinal["ROTURA"] = np.where((dfFinal["STOCK"] <= 0) & (dfFinal["PRES_STOCK"] > 0), 1, 0)

>Sinal

In [16]:
dfFinal["Sinal"]= pd.Series(dtype='int')

dfFinal['Sinal'] = np.where(dfFinal['NinjaInfo'].astype(bool) & dfFinal['SOH'].astype(bool), "Verdadeiramente Correcto",
                   np.where(~dfFinal['NinjaInfo'].astype(bool) & dfFinal['SOH'].astype(bool), "Falso Positivo",
                   np.where(dfFinal['NinjaInfo'].astype(bool) & ~dfFinal['SOH'].astype(bool), "Falso Negativo", 
                   "Rejeição Correcta")))

# 1: Há Stock e Sinal Positivo Ninja
# 2: Há Stock e Sinal Negativo Ninja
# 3: Não há Stock e Sinal Positivo Ninja
# 4: Não há Stock e Sinal Negativo Ninja

> Ciclos

In [17]:
dfFinal["Ciclos"]=dfFinal["SOH"]/dfFinal["PRES_STOCK"]

> Adequação

In [18]:
dfFinal["Adequação"] = np.where(
                        dfFinal["Ciclos"] > 1.1,
                        "Stock Suficiente",
                       np.where(
                        (dfFinal["Ciclos"] <= 1.1) & (dfFinal["INTRANSIT"] + dfFinal["EXPECTED"] + dfFinal["SOH"] >= dfFinal["PRES_STOCK"]),
                        "Stock Insuf c Forn Adequado",
                       np.where(
                        (dfFinal["Ciclos"] <= 1.1) & (dfFinal["INTRANSIT"] + dfFinal["EXPECTED"] + dfFinal["SOH"] < dfFinal["PRES_STOCK"]),
                        "Stock Insuf c Forn Desadequado",
                        ""
        )
    )
)


> Dias para a rotura de stock

In [ ]:
dfFinal["Dias_para_Rotura"] = dfFinal["Stock"] / dfFinal["SELLOUT"]

> Dias para a rotura de prateleira

In [ ]:
dfFinal["Dias_para_Rotura_Linear"] = dfFinal["PRES_STOCK"] / dfFinal["SELLOUT"]

- Roturas Linear (Ninjas)

In [28]:
#1 Quando detectada rotura
dfFinal["Roturas_reais_dia"] = dfFinal[dfFinal['NinjaInfo'] == 0].groupby(['STORE', 'DATA', 'DESC_ARTIGO'])['NinjaInfo'].transform('count')
#0 Quando não há rotura
dfFinal["Roturas_reais_dia"] = np.where(dfFinal['NinjaInfo'] == 1, 0, dfFinal['Roturas_reais_dia'])

# Acabamos com:
- Informação Ninjas
- Informação Stocks, Linear, Intransit e Expected
- Informação Sellouts, Vendedor, Reposição
- Rotura
- Sinal 
- Ciclos
- Adequação de Stock


# Reomear e organizar

# Escrever

In [36]:
df_columns = dfFinalCorr.columns.tolist()
indexed_columns = [f"{index}: {column}" for index, column in enumerate(df_columns)]
indexed_columns

['0: DATA',
 '1: Hora',
 '2: Altura_do_Dia',
 '3: Dia',
 '4: Semana',
 '5: DESC_ARTIGO',
 '6: EAN',
 '7: Nome da Loja',
 '8: STORE',
 '9: NinjaInfo',
 '10: SOH',
 '11: INTRANSIT',
 '12: EXPECTED',
 '13: PRES_STOCK',
 '14: Dia_Actual',
 '15: 1_Dia_Antes',
 '16: Reposição',
 '17: Rotura',
 '18: Sinal',
 '19: Ciclos',
 '20: Adequação',
 '21: Restock',
 '22: Roturas_reais_dia',
 '23: Roturas_Consecutivas_dia',
 '24: Roturas_Consecutivas_fds',
 '25: Proporção_Consecutivas_dia',
 '26: Std_Consecutivas_dia',
 '27: Proporção_Consecutivas_fds',
 '28: Std_Consecutivas_fds']

In [33]:
dfFinal=dfFinal.rename(columns={"VND (D-1)": "1_Dia_Antes", "Quant": "Sellout_actual"})
dfFinalCorr=dfFinal.iloc[:, [0,4,3,15,14,2,7,5,1,6,8,9,10,11,12,13,16,17,18,19,20,21,22,23,24,25,26,27,28]]

In [84]:
escrever_excel(dfFinalCorr, "OUT_1_DeltaFusion")